In [127]:
import time
import email
import smtplib
import imaplib
import mailparser
import mysql
import mysql.connector


conn=mysql.connector.connect(host="localhost",user="root",password="abcd@1234")
crs=conn.cursor()
crs.execute('use mpr')
insert='insert into db values(null,%s,%s,%s,%s,%s,%s,%s,%s)'

def read_email_from_gmail(crs):
    try:
        server = imaplib.IMAP4_SSL('imap.gmail.com')
        server.login("garglakshay631@gmail.com","abcd@1234")
        server.select('inbox')

        typ, data = server.search(None, 'ALL')
        print("Connection",typ)
        print(data)
        mail_ids = data[0]
        
        id_list = mail_ids.split()  
        first_email = int(id_list[0])
        latest_email = int(id_list[-1])
        for i in range(latest_email,first_email-1,-1):
            typ, data = server.fetch(str(i), '(RFC822)' )
            for response_part in data:
                if isinstance(response_part, tuple):
                    mail = mailparser.parse_from_bytes(response_part[-1])
                    name=mail.subject
                    to=mail.to[-1][-1]
                    from_=mail.from_[-1][-1]
                    att=mail.attachments
                    body=mail.text_plain[-1]
                    b=body.split('\r\n')
                    contact=b[-1]
                    date=b[-3]
                    venue=b[-5]
                    reg=b[-7]
                    str1=" ".join(b[:-7])
                    val=(from_,to,name,str1,date,venue,reg,contact)
                    crs.execute(insert,val)
        conn.commit()

    except (Exception, e):
        print (str(e))

In [128]:
read_email_from_gmail(crs)

Connection OK
[b'1']
